### 確率的勾配降下法とは     
データ全てを使うのではなく，全てのデータからランダムサンプリングして，取り出したデータに対して勾配を計算する

In [ ]:
import torch
import torch.nn  as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import time
from torch import optim
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook
from importnb import imports
with imports ("ipynb"):
    import  knock81 as k81

In [ ]:
# 損失の正解率の計算
def calc_loss_and_acc(model, dataset, device=None, criterion=None):
    """
    model: 評価対象のPyTorchモデル
    dataset: 評価に使うデータセット
    device: モデルやデータを転送するデバイス(CPU, GPUなど)
    criterion: 使用する損失関数
    """
    model.eval()#評価モードをオン
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)#データセットからバッチごとに取り出す
    loss = 0
    total = 0
    correct = 0
    with torch.no_grad():#no_grad: 勾配計算を不可にする(メモリの節約ができる!!)
        for d in dataloader:
            inputs = d["inputs"].to(device)#バッチのデータをデバイスの転送
            labels = d["labels"].to(device)
            outputs = model(inputs)

            #損失関数が指定されている場合，損失を計算しlossに足していく
            if criterion != None:
                loss += criterion(outputs, labels).item()
            
            pred = torch.argmax(outputs, dim=-1)#outputsの中から最高スコアのクラスを予測
            total += len(inputs)
            correct += (pred == labels).sum().item()
    
    #損失，正解率を返す
    return loss / len(dataset), correct / total

In [ ]:
# 学習する
def train_model(dataset_train, dataset_valid, batch_size, model,
                 criterion, optimizer, num_epochs, device=None):
    """
    dataset_(train,valid): 各データセット
    batch_size: バッチサイズ
    model: トレーニング対象のPyTorchモデル
    criterion: 損失関数
    optimizer: オプティマイザー
    num_epochs: エポック数
    device: モデルやデータを転送するデバイス(CPU, GPUなど)
    """
    model.to(device)#モデルをデバイスに転送
    #trainグデータ用のデータローダーを作成する
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    #validデータ用のデータローダーを作成する
    dataloader_valid = DataLoader(dataset_valid, batch_size=1, shuffle=False)

    log_train = []
    log_valid = []

    for epoch in range(num_epochs):
        start_time = time.time()

        model.train()#モデルをトレーニングモードに
        loss_train = 0.0
        for data in dataloader_train:
            optimizer.zero_grad()#勾配の初期化
            #デバイスに転送
            inputs = data["inputs"].to(device)
            labels = data["labels"].to(device)
            outputs = model.forward(inputs)#モデルにinputsを渡す
            loss = criterion(outputs, labels)#損失計算
            loss.backward()#損失の逆伝播
            optimizer.step()#オプティマイザのステップを実行
        end_time = time.time()

        model.eval()#モデルを評価モードに
        loss_train, acc_train = calc_loss_and_acc(
            model, dataset_train, device, criterion=criterion
        )
        loss_valid, acc_valid = calc_loss_and_acc(
            model, dataset_valid, device, criterion=criterion
        )
        
        #結果をlog_...に保存
        log_train.append([loss_train, acc_train])
        log_valid.append([loss_valid, acc_valid])

        #モデルやオプティマイザの状況を保存
        torch.save({"epoch":epoch, "model_statr_dict":model.state_dict(),
                    "optimizer_state_dict":optimizer.state_dict()}, f"./k82_saves/checkpoint_{epoch+1}.pt")
        
        print(f"epoch: {epoch+1},"
                f"loss_train: {loss_train:.4f},"
                f"accuracy_train: {acc_train:.4f},"
                f"loss_valid: {loss_valid:.4f},"
                f"accuracy_valid: {acc_valid:.4f},"
                f"train_time: {(end_time - start_time):.4f}sec")
        
    return {
        "train": log_train,
        "valid": log_valid
    }

In [ ]:
#　損失，正解率の可視化
def make_graph_by_logs(log):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(np.array(log['train']).T[0], label='train')
    ax[0].plot(np.array(log['valid']).T[0], label='valid')
    ax[0].set_xlabel('epoch')
    ax[0].set_ylabel('loss')
    ax[0].legend()
    ax[1].plot(np.array(log['train']).T[1], label='train')
    ax[1].plot(np.array(log['valid']).T[1], label='valid')
    ax[1].set_xlabel('epoch')
    ax[1].set_ylabel('accuracy')
    ax[1].legend()
    plt.show()  # グラフを表示
    plt.savefig("./graphs/knock82.png")  # グラフを保存

In [ ]:
VOCAB_SIZE = k81.VOCAB_SIZE
EMB_SIZE = k81.EMB_SIZE
PADDING_IDX = k81.PADDING_IDX
OUTPUT_SIZE = k81.OUTPUT_SIZE
HIDDEN_SIZE = k81.HIDDEN_SIZE
LEARNING_RATE = 1e-3
BATCH_SIZE = 1
NUM_EPOCHS = 10

model = k81.RNN(HIDDEN_SIZE, VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

log = train_model(k81.dataset_train, k81.dataset_valid, BATCH_SIZE,
                    model, criterion, optimizer, NUM_EPOCHS)
make_graph_by_logs(log)

_, acc_train = calc_loss_and_acc(model, k81.dataset_train)
_, acc_test = calc_loss_and_acc(model, k81.dataset_test)
print(f"train正解率: {acc_train:.4f}")
print(f"test正解率: {acc_test:.4f}")